
<div dir=rtl >
<h1>قوانین انجمنی</h1>

قوانین انجمنی روابط و وابستگی‌های متقابل بین مجموعه بزرگی از اقلام داده‌ای را نشان می‌دهند. 

مثال متداول در رابطه با کشف قوانین انجمنی "تحلیل سبد خرید" است.  در این فرایند با توجه به اقلام مختلفی که مشتریان در سبد خریدشان قرار می‌دهند، عادات و رفتار خرید مشتریان مورد تحلیل قرار می‌گیرد و می‌توان با شناسایی ارتباط بین محصولات، الگوهای تکرار شونده در حین خرید را بدست آورد.

سه پارامتر مهم:
- Support  میزان محبوبیت یک مجموعه آیتم را با توجه به دفعات حضور در تراکنش‌ها نشان می‌دهد.
- Confidence  احتمال خرید آیتم y در صورت خرید آیتم x را نشان می‌دهد. x -> y
- Lift   ترکیب دو پارامتر بالا می‌باشد.

برای پیاده سازی قوانین انجمنی در این تمرین، از الگوریتم Apriori که یکی از محبوب‌ترین و کارآمدترین الگوریتم‌های این حوزه است استفاده می‌کنیم.


<font color='Green'> پرسش:  تاثیر مقادیر مختلف پارامتر Lift را در احتمال وقوع نتیجه بررسی کنید.  </font>


پارامتر lift مقدار اهمیت یک قانون را بیان می‌کند و در واقع می‌توان گفت نسبت اطمینان یک قانون به اطمینان مورد انتظار را بیان می‌کند.
همانطور که بررسی کردیم اگر مقدار lift را بزرگتر از یک قرار بدهیم در واقع باعث می‌شود که وقوع بدنه قانون تاثیر مثبتی بر وقوع قسمت اول قانون موردنظر داشته باشد. در واقع بدنه قانون و قسمت اول قانون بیشتر از حد انتظار باهم ظاهر می‌شوند
    و درصورتی که کوچکتر از یک باشد بدنه قانون و قسمت اول قانون کمتر از حد مورد انتظار باهم ظاهر می‌شوند.
    
</div>





<div dir=rtl >
<h1>الگوریتم Apriori</h1>

روش کار الگوریتم به این صورت است که یک  حداقل مقدار پشتیبانی در نظر گرفته    می‌شود و تکرارها با مجموعه آیتم‌های مکرر اتفاق می‌افتند. در صورتیکه  مجموعه‌ها و زیرمجموعه‌ها مقدار پشتیبانی کمتر از حد آستانه داشته باشند حذف می‌شوند. این روند تا زمانیکه امکان حذف وجود نداشته باشد ادامه می‌یابد. 

در این بخش از تمرین می‌خواهیم الگوریتم apriori را بر روی مجموعه داده Hypermarket_dataset که شامل سفارشات خرید افراد از فروشگاه های مواد غذایی است اعمال کنیم.


</div>





<div dir=rtl >
<h1>آماده‌سازی داده</h1>

<font color='Green'>پرسش: برای شروع کار باید داده‌های دیتاست را در قالب ماتریس پراکنده که محصولات  خریداری شده در ستون‌ و شماره سفارشات خرید به عنوان شاخص (index) هستند آماده کنید. 


برای راحتی کار محصولات خریداری شده در هر سفارش را با اعداد 0,1 کدگذاری کنید.

 نمونه ماتریس خروجی شما:

<img src="https://drive.google.com/uc?id=1eD0jan1ZbeYqSklgK--ks7oeY-MyTA3p"></img>

</div>




In [9]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
 
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [10]:
data = pd.read_csv("Hypermarket_dataset.csv")
data.head()
data.itemDescription = data.itemDescription.transform(lambda x: [x])
data = data.groupby(['Member_number']).sum()['itemDescription'].reset_index(drop=True)
encoder = TransactionEncoder()
transactions = pd.DataFrame(encoder.fit(data).transform(data).astype("int"), columns=encoder.columns_)
transactions.head()

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,1,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0



<div dir=rtl >
<h1>شناسایی الگوهای پر تکرار</h1>

<font color='Green'> پرسش:  با اعمال الگوریتم apriori و به ازای مقدار min_support = 0.07 تمامی الگوهای پر تکرار را تولید نمایید. 

</div>




In [11]:
frequent_itemsets = apriori(transactions, min_support= 0.07, use_colnames=True)
frequent_itemsets

/usr/local/Cellar/jupyterlab/3.3.2/libexec/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.078502,(UHT-milk)
1,0.119548,(beef)
2,0.079785,(berries)
3,0.158799,(bottled beer)
4,0.213699,(bottled water)
...,...,...
78,0.075680,"(yogurt, tropical fruit)"
79,0.079785,"(whole milk, whipped/sour cream)"
80,0.150590,"(whole milk, yogurt)"
81,0.082093,"(rolls/buns, other vegetables, whole milk)"



<div dir=rtl >
<h1>استخراج قوانین انجمنی</h1>


<font color='Green'> پرسش: تابعی بنویسید که با گرفتن دو ورودی confidence و lift قوانین انجمنی حاصل را در خروجی نشان دهد. </br>
برای دو حالت خروجی خود را در گزارش ثبت کنید.


</div>




In [12]:
rules = association_rules(frequent_itemsets , metric="lift")

def rules_construction(confidence , lift):
    filter = (rules['confidence'] >= confidence) & (rules['lift'] >= lift)
    filter_rules = rules[filter]
    return filter_rules
rules_1 = rules_construction(0.4, 1.2)
display(rules_1)
rules_2 = rules_construction(0.5, 0.8)
display(rules_2)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
86,"(rolls/buns, other vegetables)",(whole milk),0.146742,0.458184,0.082093,0.559441,1.220996,0.014859,1.229837
87,"(rolls/buns, whole milk)",(other vegetables),0.178553,0.376603,0.082093,0.459770,1.220834,0.014850,1.153947
88,"(other vegetables, whole milk)",(rolls/buns),0.191380,0.349666,0.082093,0.428954,1.226753,0.015174,1.138847
92,"(whole milk, yogurt)",(other vegetables),0.150590,0.376603,0.071832,0.477002,1.266589,0.015119,1.191967
94,"(yogurt, other vegetables)",(whole milk),0.120318,0.458184,0.071832,0.597015,1.303003,0.016704,1.344507


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(bottled beer),(whole milk),0.158799,0.458184,0.085428,0.537964,1.174124,0.012669,1.172672
9,(bottled water),(whole milk),0.213699,0.458184,0.112365,0.525810,1.147597,0.014452,1.142615
11,(canned beer),(whole milk),0.165213,0.458184,0.087224,0.527950,1.152268,0.011526,1.147795
19,(domestic eggs),(whole milk),0.133145,0.458184,0.070292,0.527938,1.152242,0.009287,1.147766
21,(newspapers),(whole milk),0.139815,0.458184,0.072345,0.517431,1.129310,0.008284,1.122775
39,(other vegetables),(whole milk),0.376603,0.458184,0.191380,0.508174,1.109106,0.018827,1.101643
43,(pastry),(whole milk),0.177527,0.458184,0.091072,0.513006,1.119651,0.009732,1.112572
45,(pip fruit),(whole milk),0.170600,0.458184,0.086968,0.509774,1.112598,0.008801,1.105239
54,(rolls/buns),(whole milk),0.349666,0.458184,0.178553,0.510638,1.114484,0.018342,1.107190
67,(sausage),(whole milk),0.206003,0.458184,0.106978,0.519303,1.133394,0.012591,1.127146
